In [1]:
# pip install torchtext spacy
# python3 -m spacy download pt_core_news_sm

In [1]:
import torch
import torchtext
from torchtext.legacy import data

device = torch.device("cpu")

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, drop_prob, hs1, num_class, freeze=False):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        if freeze:
            self.embedding.weight.requires_grad = False
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(embed_dim, hs1) 
        self.fc2 = nn.Linear(hs1, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = embedded.view(embedded.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        preds = self.softmax(self.fc2(x))
        return preds

In [3]:
text = data.Field()

# Criando dataset de treino e de teste
data = data.TabularDataset(path="df_dataset.csv",
                                 format="csv",
                                 fields=[
                                        ('', text),
                                        ('', text),
                                        ('has_anger', data.Field()),
                                        ('', text),
                                        ('txt', text)
                                 ],
                                 skip_header=True)

# train_data = data.TabularDataset(path="df_dataset.csv",
#                                  format="csv",
#                                  fields=[
#                                         ('', text),
#                                         ('', text),
#                                         ('has_anger', data.Field()),
#                                         ('', text),
#                                         ('txt', text)
#                                  ],
#                                  skip_header=True)

# test_data = data.TabularDataset(path="df_dataset_test.csv",
#                                 format="csv",
#                                 fields=[
#                                         ('', text),
#                                         ('', text),
#                                         ('has_anger', data.Field()),
#                                         ('', text),
#                                         ('txt', text)
#                                 ],
#                                 skip_header=True)

#text.build_vocab(train_data, test_data)

In [4]:
# len(train_data)
len(data)

7251

In [5]:
MAX_VOCAB_SIZE = 25000

#PAD_IDX = text.vocab.stoi[text.pad_token]

In [6]:
VOCAB_SIZE = 25002
NGRAMS = 2
BATCH_SIZE = 8
EMBED_DIM = 300
NUM_CLASS = 2
DROPOUT_PROB = 0.5
HS1 = 128

model = TextSentiment(VOCAB_SIZE, EMBED_DIM, DROPOUT_PROB, HS1, NUM_CLASS, True).to(device)

In [8]:
import torchtext.vocab as vocab

pretrained_embeddings = vocab.Vectors(name = 'nilc_embeddings/fasttext_cbow_s300.txt')

text.build_vocab(data,
#                  train_data, 
#                  test_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = pretrained_embeddings,
                 unk_init = torch.Tensor.normal_)

model.embedding.weight.data.copy_(text.vocab.vectors)
# UNK_IDX = text.vocab.stoi[text.unk_token]

# model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBED_DIM)
#model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBED_DIM)

#print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 3.5951, -0.9948, -2.4219,  ...,  2.0744,  0.1685, -2.3501],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [9]:
# Word tokenization
import spacy
import string

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('pt_core_news_sm')
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

# Creating our tokenizer function
def spacy_tokenizer(sentence, lemmatize=False, remove_stop_words=False, remove_punctuations=False):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lematizando os tokens e colocando em caixa baixa
    if (lemmatize):
        mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    else:
        mytokens = [ word.text.lower().strip() for word in mytokens ]

    # Removendo stop words
    if (remove_stop_words):
        mytokens = [ word for word in mytokens if word not in stop_words ]        
        
    if (remove_punctuations):
        mytokens = [ word for word in mytokens if word not in punctuations ]
        
    # Retornando a lista de token reprocessado
    return mytokens

text_pipeline = lambda x: spacy_tokenizer(x, lemmatize=True, remove_stop_words=True)
label_pipeline = lambda x: int(x)

In [10]:
def generate_batch(batch):
    label = torch.tensor([label_pipeline(entry.has_anger[0]) for entry in batch])
    _text = []
    for entry in batch:
        _entry = []
        entry_txt = text_pipeline("".join(entry.txt))
        for t in entry_txt:
            _entry.append(text.vocab.stoi[t])
        _text.append(torch.tensor(_entry,dtype=torch.long))
    offsets = [0] + [len(entry) for entry in _text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    _text = torch.cat(_text)
    #print(' BATCH' , _text)
    #print(offsets)
    return _text, offsets, label

In [11]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        #output = model(text)
        loss = criterion(output, cls)
        train_loss += loss.item()
        # Clear the gradient buffers of the optimized parameters.
        # Otherwise, gradients from the previous batch would be accumulated.
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            #output = model(text)
            #print(output, cls)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [13]:
import time
from torch.utils.data.dataset import random_split
import torch.optim as optim
N_EPOCHS = 1
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
#optimizer = optim.SparseAdam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

# Divide 90% para o treino e 10% para os testes
train_len = int(len(data) * 0.90)
sub_train_, sub_test_ = random_split(data, [train_len, len(data) - train_len])
sub_train_

# Do que sobrou do treino, 95% para o treino e 5% para a validacao
train_len = int(len(sub_train_) * 0.95)
sub_train_, sub_valid_ = random_split(sub_train_, [train_len, len(sub_train_) - train_len])
sub_train_

6525


In [14]:

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print(f'Epoch: {epoch + 1}, | time in {mins} minutes and {secs} seconds')
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1, | time in 1.65 minutes and 39 seconds
	Loss: 0.0617(train)	|	Acc: 82.0%(train)
	Loss: 0.0028(valid)	|	Acc: 83.2%(valid)


In [15]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(sub_test_)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0013(test)	|	Acc: 83.3%(test)


In [16]:
def ngrams_iterator(token_list, ngrams):
    """Return an iterator that yields the given tokens and their ngrams.

    Arguments:
        token_list: A list of tokens
        ngrams: the number of ngrams.

    Examples:
        >>> token_list = ['here', 'we', 'are']
        >>> list(ngrams_iterator(token_list, 2))
        >>> ['here', 'here we', 'we', 'we are', 'are']
    """

    def _get_ngrams(n):
        return zip(*[token_list[i:] for i in range(n)])

    for x in token_list:
        yield x
    for n in range(2, ngrams + 1):
        for x in _get_ngrams(n):
            yield ' '.join(x)


In [26]:
NGRAMS = 2
# label = {0 : "N",
#          1 : "S"}

def predict(_text, model, vocab, ngrams):
    if len(_text) == 0:
        return 0
    with torch.no_grad():
        _text = [vocab.stoi[token] for token in ngrams_iterator(_text, ngrams)]
        output = model(torch.tensor(_text), torch.tensor([0]))
        return output.argmax(1).item()

Acertos: 604
Total: 726
Acurácia: 83.1956%
Precision: 0.7555%
Recall: 0.9689%
Fscore: 0.8490%


In [ ]:
model = model.to('cpu')

# Rodando os testes e obtendo dados da matriz de confusao para calculo das metricas
cm = {'true_positive': 0, 'true_negative': 0, 'false_positive': 0, 'false_negative': 0}
for entry in sub_test_:
    if (predict(text_pipeline("".join(entry.txt)), model, text.vocab, NGRAMS) == int(entry.has_anger[0])):
        if int(entry.has_anger[0]) == 1:
            cm['true_positive'] += 1
        else:
            cm['true_negative'] += 1
    else:
        if int(entry.has_anger[0]) == 0:
            cm['false_positive'] += 1
        else:
            cm['false_negative'] += 1

# Metricas
total = len(sub_test_)
precision = cm['true_positive'] / (cm['true_positive'] + cm['false_positive'])
recall = cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])
fscore = 2 * ((precision * recall) / (precision + recall))
acertos = cm['true_positive'] + cm['true_negative']
print(f'Acertos: {cm["true_positive"] + cm["true_negative"]}')
print(f'Total: {total}')
print(f'Acurácia: {(acertos / total) * 100:.4f}%')
print(f'Precision: {precision:.4f}%')
print(f'Recall: {recall:.4f}%')
print(f'Fscore: {fscore:.4f}%')